In [1]:
import pandas as pd
import numpy as np
import nltk
import re

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [2]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [3]:
punctuation += '№'
punctuation += '®'
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~№®'

In [4]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maximmezhov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
custom_top_words = ['мл','мг','шт','ooo','ру','лп','н','xch','bky','уп','%','п','изм','нд','p','р','n','\n','b','e','е','г','упаковка','пачка','серия','партия','рег','лср',
                    'pul',':','инвойс','rus','вэд','+','rg','g','ltd','d','c','д','кг','-','tyd','дата','оао','ао','зао','ип','инн','лс','код','окпд','россия','ул','область']
numbers = [i for i in range(10)]
numbers = list(map(str, numbers))

In [6]:
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")
eng_stopwords = stopwords.words("english")

In [7]:
data = pd.read_excel('data/raw/Реестр 327 тыс. деклараций ЕП РФ без 140000-200000.xlsx', engine='openpyxl')
data.head()

,id,Общее наименование продукции,Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции),Подкатегория продукции
0,1,"Парацетамол таблетки 500 мг 10 шт., упаковки я...",9300.1,"Лекарственные средства, зарегистрированные в у..."
1,2,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...
2,3,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...
3,4,Аппарат вакуумно-лазерной терапии стоматологич...,9444.4,Приборы и аппараты для электролечения высокоча...
4,5,Блоки оконные и балконные дверные из алюминиев...,5270.1,Блоки оконные и балконные дверные из алюминиев...


In [8]:
def return_right_code(codes):
    """
    codes - строка нескольких кодов (тип list)
    """
    arr = np.array(codes.split('; '), dtype=np.float)
    arr2 = np.array(list(map(int, arr)))
    
    if sum(arr2/arr2.min()) == arr.shape:
        """
        если коды одной группы, то берём максимальное значение подгруппы
        """
        code = arr.max()
    else:
        """
        коды разной группы, возвращаем 0
        """
        code = 0

    return code
    
    
def preprocess_text(text):
    """
    Очистка и лемматизация текста
    Присутствует специфика данных
    """
    tokens = mystem.lemmatize(text.lower())
    
    text = [re.sub('[0-9][а-я]|\d+', '*', token) for token in tokens if (token.strip() not in russian_stopwords) & \
            (token.strip() not in eng_stopwords) & \
            (token.strip() not in punctuation) & \
            (token.strip() not in numbers) & \
            (token.strip() not in custom_top_words) & \
            (token != ' ')]
    if '*' in text:
        text.remove('*')
        
    text = ' '.join(text)
    
    text = text.replace('*', '')
    text = text.replace('®', '')
    text = text.replace('№', '')
    text = text.replace(')', '')
    text = text.replace('(', '')
    text = text.replace('.', '')
    text = text.replace('"', '')
    text = text.replace('\n', '')
    text = text.replace('«', '')
    text = text.replace('»', '')
    text = text.replace(',,', '')
    text = text.replace(',', '')
    text = text.replace('окпд', '')
    text = text.replace('ooo', '')  # eng
    text = text.replace('ооо', '')  # rus
    text = text.split(' ')
    text = ' '.join([x for x in text if x not in ['']])
    
        
    return text


def preprocessing_raw_data(data):
    """
    Предобработка превоначального набора данных
    """
    df = data.copy()
    df.drop(columns=['id','Подкатегория продукции'], inplace=True, axis=1)
    df.rename({'Общее наименование продукции':'description', 'Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции)':'code'}, axis=1, inplace=True)
    
    """
    преобразование кодов:
    - если есть категория и подкатегория - оставляем подкатегорию (так понял эксперта на чек-поинте)
    - если есть несколько категорий/подкатегорий, которые отличаются своей основной категорией - принимаем за махинацию, 
                                                                                                 отбираем в отдельный массиив для ручного реагирования, 
                                                                                                 в обучающую выборку не берём
    """
    df['processed-code'] = df['code'].apply(return_right_code)
    df.drop(index=df[df['processed-code'] == 0].index, inplace=True)
    
    df.code = df['processed-code']
    df.drop('processed-code', axis=1, inplace=True)
    
    
    """
    предобработка текста
    """
    df['clear-description'] = df['description'].apply(preprocess_text)
    
    
    return df[['clear-description','code','description']]

In [231]:
%%time
df = preprocessing_raw_data(data.iloc[:30,:])
df.to_csv('data/processed/test.csv',index=False)
df.head()

CPU times: user 20 ms, sys: 2.81 ms, total: 22.8 ms
Wall time: 40.4 ms


,clear-description,code,description
0,парацетамол таблетка ячейковый контурный карто...,9300.1,"Парацетамол таблетки 500 мг 10 шт., упаковки я..."
1,перезаряжать литий ионный батарея торговый мар...,3482.2,Перезаряжаемая литий-ионная батарея торговой м...
2,перезаряжать литий ионный батарея торговый мар...,3482.2,Перезаряжаемая литий-ионная батарея торговой м...
3,аппарат вакуумный лазерный терапия стоматологи...,9444.4,Аппарат вакуумно-лазерной терапии стоматологич...
4,блок оконный балконный дверной алюминиевый про...,5270.1,Блоки оконные и балконные дверные из алюминиев...


In [232]:
df.loc[0,'clear-description']

'парацетамол таблетка ячейковый контурный картонный годный производство фармстандарт лексредство курская курск агрегатный'

In [9]:
df = preprocessing_raw_data(data)
df.to_csv('data/processed/precessed-dataframe.csv', index=False)

In [ ]:
!python3 data-preprocessing.py

In [ ]:
data.shape, df.shape

((267684, 4), (267588, 2))

In [122]:
df['code'].value_counts()

9300.1    157928
2364.1     10799
3482.2      4654
5990.1      3802
9392.1      3698
           ...  
9435.0         1
9434.0         1
2322.1         1
2553.0         1
3454.6         1
Name: code, Length: 396, dtype: int64

In [103]:
# df.loc[df['processed-code'] == 0, 'code'].tolist()

['2386.1; 9392.2',
 '2386.1; 9392.2',
 '2293.2; 5463.8',
 '9444.10; 9398.5',
 '3482.1; 3483.1',
 '9436.1; 2545.2',
 '5463.8; 2381.2',
 '9433.3; 9433.2; 9433.1; 9432.1; 9433.5; 9435.1; 9434.1',
 '5970.1; 2293.3',
 '3414.16; 3412.1',
 '2293.3; 5970.1',
 '5990.1; 1482.1',
 '1482.1; 1483.1',
 '2386.3; 9392.1',
 '2293.5; 1482.1',
 '9434.1; 9433.1; 9435.1',
 '9432.1; 2514.5; 9433.1; 5463.8',
 '2293; 2380.1',
 '9398.2; 9436.1',
 '2440.1; 2387.1',
 '2440.1; 2387.1',
 '2387.1; 2440.1',
 '2387.1; 2440.1',
 '2440.1; 2387.1',
 '2387.1; 2440.1',
 '9437.1; 9396.1',
 '1483.1; 1482.1',
 '2440.1; 2387.1',
 '2387.1; 2440.1',
 '1482.1; 1483.1',
 '9439.1; 9431.1',
 '9391.1; 9450.7',
 '9296.5; 9146.1',
 '9146.1; 9296.5',
 '9451.1; 9450.3',
 '1483.1; 1482.1',
 '3482.2; 3481.1',
 '3483.1; 2293.1',
 '1483.1; 1482.1',
 '9396.9; 9432.1',
 '9436.1; 9441.4',
 '9441.4; 9436.1',
 '1482.1; 1483.1',
 '2293; 1482',
 '1481.1; 5982.1; 1482.1',
 '9393.1; 9439',
 '9437.1; 9436.1',
 '9290.1; 9296.3',
 '3414.14; 9441.11',
 

In [106]:
# df[df['processed-code'] == 0].drop('processed-code', axis=1).to_csv('frod-docs.csv', index=False)